<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Clasificación supervisada Random Forest
Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [1]:
# Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print("Installing geemap ...")
#     subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

In [2]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Crear un mapa interactivo 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Escogemos la imagen landsat 

In [4]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")


In [5]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
clasificacion_18c = ee.Image("users/franciscoalbornoz/clasificacion_18c")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(clasificacion_18c.randomVisualizer(), {}, "clasificación 2018")
Map

Map(center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Importar rasters locales 

In [6]:
%pwd

'/mnt/d/Francisco Albornoz/Github/VitoAlbornoz/GEE'

In [7]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 

In [8]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [9]:
out_dir

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data'

In [10]:
raster_aysen

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [11]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')
Map

The image file does not exist.


Map(center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [12]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [13]:
# raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [14]:
# raster.bounds

In [15]:
# raster.crs

In [16]:
# raster

## Obteniendo los valores para el Training Datasets

In [17]:
# points = landcover.sample(**{
# #     'region': landsat.geometry(),
#     'scale': 30,
#     'numPixels': 20000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

# Map.addLayer(points, {}, 'training', False)
# Map

In [18]:
work_dir = os.path.expanduser('../Data/shapes/')


merged_shp = os.path.join(work_dir, 'roi_h.shp')
val_shp= os.path.join(work_dir, 'validacion.shp')
merged_original =os.path.join(work_dir,'roi_merged.shp')

In [19]:
merged = geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [20]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [21]:
mergedoriginal =geemap.shp_to_ee(merged_original)
Map.addLayer(mergedoriginal, {}, 'Áreas de entrenamiento Original')

In [22]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['IDH'],
    'scale' : 30 ,
})

In [23]:
dataset1 = mosaico.sampleRegions(**{
    'collection': mergedoriginal,
    'properties': ['ID'],
    'scale': 30,
})

In [24]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [25]:
dataextrac = clasificacion_18c.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [26]:
print(dataset1.size().getInfo())

14254


In [27]:
print(datasetval.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'ID': 3, 'b1': 0.01979999989271164, 'b2': 0.017999999225139618, 'b3': 0.010499999858438969, 'b4': 0.01730000041425228, 'b5': 0.00989999994635582, 'b6': 0.0071000000461936, 'b7': 0.24460433423519135, 'b8': -0.019829994067549706, 'b9': 0.019325504079461098}}


### Dividiendo en muestras y datos de validación

In [28]:
# Guardamos las bandas
band = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
# Esta propiedad guarda las clases de landcover
# label = 'b1'
label = 'IDH'
label1 ='ID'
# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# # Adds a column of deterministic pseudorandom numbers. 
# sample = sample.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [29]:
# training.first().getInfo()

In [30]:
# validation.first().getInfo()

## Entrenando el modelo 

In [31]:
classifier = ee.Classifier.smileRandomForest(10).train(dataset, label, bands)

In [32]:
classifier1 = ee.Classifier.smileRandomForest(10).train(dataset1, label1, bands)

In [33]:
classifier100 = ee.Classifier.smileRandomForest(100).train(dataset1, label1, bands)

### Clasificando la imagen

In [34]:
# Clasificando con las mismas bandas de entrenamiento
result1 = mosaico.select(bands).classify(classifier)
result2 = mosaico.select(bands).classify(classifier1)
result3 = mosaico.select(bands).classify(classifier100)
# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'classificado')
# Map

In [35]:
# result.getInfo()

### Categorizando según las clases 

In [36]:
# landcover.getInfo()

In [37]:
class_values = [1,2,3,4,5,6,7,8,9,10,11]
class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
class_palette = ['0c40f1', #1 agua*
 'f0ff00', #2 cultivo*
 'f99504', #3 estepa*
 '17d4c6', #4 humedal*
 '24ff00', #5 matorral abierto*
 '2de50f', #6 matorral arborescente*
 'ccba7d', #7 plantaciones*
 'dec61a', #8 praderas*
 '28d909', #9 primario*
 '4e8c44', #10 secundario*
 '685205', #11 suelo desnudo*
]
class_palette1 = ['22c129', # 1
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [38]:
# clasificacion_18c= ee.Image("users/franciscoalbornoz/clasificacion_18c")
# Map.addLayer(clasificacion_18c, class_palette1 , 'Land cover clasificado Random Forest Mosaico 2018' )

In [39]:
# class_values = [1,
#  2,
#  3,
#  4,
#  5,
#  6,
#  7,
#  8,
#  9,
#  10,
#  11,
#  12,
#  13,
#  14,
#  15,
#  16]

In [40]:
# class_palette = ['22c129',
#  'f0ff00',
#  '0c40f1',
#  '685205',
#  '0d4404',
#  'e82d13',
#  'f99504',
#  '17d4c6',
#  '1c6330',
#  '24ff00',
#  '2de50f',
#  'ccba7d',
#  'e3e3c2',
#  '28d909',
#  '03c2ff',
#  '4e8c44',
# ]


In [41]:
# clases = ["Bosque Adulto",       #1
#            "Renoval",             #2
#            "Matorrales",          #3
#            "Praderas",            #4
#            "Agropecuario",        #5
#            "Agua",                #6
#            "Humedales",           #7 
#            "Suelo Desnudo",       #8
#            "Plantación Forestal", #9
#            "Incendio",            #10
#            "Urbano",              #11
#            "Estepa",              #12
#            "Estepa abierta",      #13
#            "Glaciares-Nieve",     #14
#            "Vegetación de altura",#15
#            "No clasificado"]     #16

In [42]:
landcover1 = result1.set('classification_class_values', class_values)
landcover1 = landcover1.set('classification_class_palette', class_palette)

In [43]:
landcover2 = result2.set('classification_class_values', class_values1)
landcover2 = landcover2.set('classification_class_palette', class_palette1)

In [44]:
landcover3 = result3.set('classification_class_values', class_values1)
landcover3 = landcover3.set('classification_class_palette', class_palette1)

In [45]:
landcoveroriginal = result3.set('classification_class_values', class_values1)
landcoveroriginal = landcover3.set('classification_class_palette', class_palette1)

In [46]:
Map.addLayer(landcover1, {}, 'Land cover clasificado Random Forest Mosaico 2018' )

In [47]:
Map.addLayer(landcover2, {}, 'Land cover clasificado RF 10' )

In [48]:
Map.addLayer(landcover3, {}, 'Land cover clasificado RF 100' )
legend_dict = {
    '01 Achaparrado': '22c129',
    '02 Agrícola': 'f0ff00',
    '03 Agua': '0c40f1',
    '04 Arenal o Roca': '685205',
    '05 Bosque Degradado': '0d4404',
    '06 Cuncunas': 'e82d13',
    '07 Estepa': 'f99504',
    '08 Humedal': '17d4c6',
    '09 Mallin': '1c6330',
    '10 Matorral abierto': '24ff00',
    '11 Matorral denso': '2de50f',
    '12 Plantaciones': 'ccba7d',
    '13 Praderas': 'dec61a',
    '14 Primario': '28d909',
    '15 Ríos': '03c2ff',
    '16 Secundario': '4e8c44',
    '17 Sombra topo': 'fdfdfd',
    '18 Suelo desnudo': 'c0c233'
}
Map.add_legend(legend_title="Random Forest", legend_dict=legend_dict)
Map


Map(center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Visualizando el resultado


In [49]:
# print('Change layer opacity:')
# cluster_layer = Map.layers[-1]
# cluster_layer.interact(opacity=(0, 1, 0.1))

### Agregando título al mapa

In [50]:
# Map.add_legend('Clasificación supervisada Random Forest')
# Map

### Evaluación de presición / Accuracy assessment

In [51]:
train_accuracy = classifier1.confusionMatrix()

In [52]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 624, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 806, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 3020, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1622, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 97, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 930, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 4, 0, 0, 617, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 48, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 222, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 7, 0, 0, 0, 2, 150, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3097, 0, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 252, 0, 0, 0, 0, 2],
 [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 779, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 217, 0, 0, 0],
 [0, 7, 0,

In [53]:
train_accuracy.accuracy().getInfo()

0.9972639259155325

In [54]:
train_accuracy.kappa().getInfo()

0.9968688213363049

In [55]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9984],
 [1],
 [1],
 [0.9993838570548367],
 [0.97],
 [0.9989258861439313],
 [0.9935587761674718],
 [0.9795918367346939],
 [1],
 [1],
 [0.9259259259259259],
 [0.999677211103938],
 [0.9767441860465116],
 [0.9974391805377721],
 [1],
 [0.9897058823529412],
 [1],
 [1]]

In [56]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.985781990521327,
  0.9950617283950617,
  0.9996689837802052,
  0.9969268592501537,
  0.9897959183673469,
  0.9893617021276596,
  0.9983818770226537,
  1,
  1,
  0.9910714285714286,
  0.9868421052631579,
  1,
  1,
  0.9974391805377721,
  1,
  1,
  1,
  0.9975669099756691]]

Random Forest 100

In [57]:
train_accuracy1 = classifier100.confusionMatrix()

In [58]:
train_accuracy1.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 625, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 806, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 3020, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1622, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 931, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 621, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 221, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 161, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3097, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 258, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 781, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 217, 0, 0, 0],
 [0, 1, 0

In [59]:
train_accuracy1.accuracy().getInfo()

0.9996492212712221

In [60]:
train_accuracy1.kappa().getInfo()

0.9995986061569231

In [61]:
train_accuracy1.producersAccuracy().getInfo()

[[0],
 [1],
 [1],
 [1],
 [0.9993838570548367],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0.9954954954954955],
 [0.9938271604938271],
 [0.999677211103938],
 [1],
 [1],
 [1],
 [0.9985294117647059],
 [1],
 [1]]

In [62]:
train_accuracy1.consumersAccuracy().getInfo()

[[0,
  0.9984025559105432,
  0.9987608426270136,
  1,
  1,
  1,
  0.9978563772775991,
  1,
  1,
  1,
  1,
  0.9938271604938271,
  1,
  1,
  1,
  1,
  1,
  1,
  1]]

#### Set de datos de validación

In [63]:
validated = datasetval.classify(classifier1)

In [64]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'ID': 3,
  'b1': 0.01979999989271164,
  'b2': 0.017999999225139618,
  'b3': 0.010499999858438969,
  'b4': 0.01730000041425228,
  'b5': 0.00989999994635582,
  'b6': 0.0071000000461936,
  'b7': 0.24460433423519135,
  'b8': -0.019829994067549706,
  'b9': 0.019325504079461098,
  'classification': 3}}

In [65]:
test_accuracy = validated.errorMatrix('ID', 'classification')

In [66]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 607, 0, 0, 8, 0, 0, 3, 0, 0, 7, 0, 0, 0, 0, 4],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 51, 0, 0, 204, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 80],
 [0, 0, 0, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 5, 24, 0, 10, 14, 0, 0, 0, 0, 20, 21, 15, 12, 6, 0, 0, 3, 12],
 [0, 43, 13, 0, 0, 17, 6, 0, 0, 0, 7, 64, 12, 2, 80, 0, 23, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 6, 0, 0, 0, 0],
 [0, 0, 127, 0, 0, 2, 0, 8, 0, 0, 7, 12, 0, 66, 0, 0, 0, 0, 69],
 [0, 445, 22, 0, 8, 7, 14, 0, 0, 0, 0, 17, 19, 0, 408, 0, 160, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 

In [67]:
test_accuracy.accuracy().getInfo()

0.5127542783338714

In [68]:
test_accuracy.kappa().getInfo()

0.4406587228910788

In [69]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.926829268292683],
 [0.8627450980392157],
 [0.9650238473767886],
 [0.75],
 [0],
 [0.6035502958579881],
 [0],
 [0.5],
 [0.14084507042253522],
 [0.2397003745318352],
 [0.8125],
 [0.2268041237113402],
 [0.3709090909090909],
 [0],
 [0.6341463414634146],
 [0],
 [0]]

In [70]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.005836575875486381,
  0.168141592920354,
  0.9777777777777777,
  0.8659058487874465,
  0.06976744186046512,
  0,
  0.9272727272727272,
  0,
  1,
  0.5405405405405406,
  0.5565217391304348,
  0.33766233766233766,
  0.7333333333333333,
  0.758364312267658,
  0,
  0.3623693379790941,
  0,
  0]]

RF 100 Validación

In [71]:
validated1 = datasetval.classify(classifier100)

In [72]:
validated1.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'ID': 3,
  'b1': 0.01979999989271164,
  'b2': 0.017999999225139618,
  'b3': 0.010499999858438969,
  'b4': 0.01730000041425228,
  'b5': 0.00989999994635582,
  'b6': 0.0071000000461936,
  'b7': 0.24460433423519135,
  'b8': -0.019829994067549706,
  'b9': 0.019325504079461098,
  'classification': 3}}

In [73]:
test_accuracy1 = validated1.errorMatrix('ID', 'classification')

In [74]:
test_accuracy1.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 610, 0, 0, 5, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 35, 0, 0, 202, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 93],
 [0, 0, 0, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 6, 16, 0, 8, 14, 0, 0, 0, 0, 16, 35, 6, 21, 12, 0, 0, 3, 5],
 [0, 36, 15, 0, 0, 7, 5, 0, 0, 0, 2, 81, 11, 2, 88, 0, 20, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 0, 5, 0, 0, 0, 0],
 [0, 0, 125, 0, 0, 0, 0, 8, 0, 0, 7, 14, 0, 66, 0, 0, 0, 0, 71],
 [0, 464, 32, 0, 8, 1, 8, 0, 0, 0, 0, 31, 12, 0, 408, 0, 128, 8, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,

In [75]:
test_accuracy1.accuracy().getInfo()

0.5175976751695189

In [76]:
test_accuracy1.kappa().getInfo()

0.44553052775835783

In [77]:
test_accuracy1.producersAccuracy().getInfo()

[[0],
 [1],
 [0.926829268292683],
 [0.8627450980392157],
 [0.9697933227344993],
 [1],
 [0],
 [0.5976331360946746],
 [0],
 [0.5],
 [0.11267605633802817],
 [0.30337078651685395],
 [0.84375],
 [0.2268041237113402],
 [0.3709090909090909],
 [0],
 [0.6280487804878049],
 [0],
 [0]]

In [78]:
test_accuracy1.consumersAccuracy().getInfo()

[[0,
  0.005671077504725898,
  0.16666666666666666,
  0.9777777777777777,
  0.8892128279883382,
  0.15384615384615385,
  0,
  0.9395348837209302,
  0,
  1,
  0.64,
  0.5031055900621118,
  0.42857142857142855,
  0.5945945945945946,
  0.7445255474452555,
  0,
  0.4103585657370518,
  0,
  0]]

In [79]:
# validated100 = dataextrac.classify(classifier100)

In [80]:
# validated100.first().getInfo()

In [81]:
# dataextrac.first().getInfo()

In [82]:
# test_accuracy100 = dataextrac.errorMatrix('ID', 'b1')

In [83]:
# test_accuracy100.getInfo()

In [84]:
# test_accuracy100.accuracy().getInfo()

In [85]:
# test_accuracy100.kappa().getInfo()

In [86]:
# test_accuracy100.producersAccuracy().getInfo()

In [87]:
# test_accuracy100.consumersAccuracy().getInfo()

### Descargando la matrix

In [88]:
# import csv
# import os

# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/'), 'Desktop')
# training_csv = os.path.join(out_dir, 'train_accuracy.csv')
# testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())
    
# with open(testing_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(test_accuracy.getInfo())

### Exportando 

In [89]:
# import os
# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/'), 'Desktop')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [90]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)


In [91]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)